In [5]:
import pandas as pd
import matplotlib.pyplot as pyplot
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Setting to make numbers easier to read on display
pd.options.display.float_format = '{:20,.2f}'.format

#Show all columns on output
pd.set_option('display.max_columns', 999)


## Data Exploration

In [6]:
df = pd.read_excel("../data/online_retail_II.xlsx", sheet_name=0)

df.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,"13,085.00",United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,"13,085.00",United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,"13,085.00",United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,"13,085.00",United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,"13,085.00",United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,"13,085.00",United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,"13,085.00",United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,"13,085.00",United Kingdom
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,"13,085.00",United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,"13,085.00",United Kingdom


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [8]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,"525,461.00",525461,"525,461.00","417,534.00"
mean,10.34,2010-06-28 11:37:36.845017856,4.69,"15,360.65"
min,"-9,600.00",2009-12-01 07:45:00,"-53,594.36","12,346.00"
25%,1.00,2010-03-21 12:20:00,1.25,"13,983.00"
50%,3.00,2010-07-06 09:51:00,2.10,"15,311.00"
75%,10.00,2010-10-15 12:45:00,4.21,"16,799.00"
max,"19,152.00",2010-12-09 20:01:00,"25,111.09","18,287.00"
std,107.42,NaN,146.13,"1,680.81"


In [9]:
df.describe(include='O')

,Invoice,StockCode,Description,Country
count,525461,525461,522533,525461
unique,28816,4632,4681,40
top,537434,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
freq,675,3516,3549,485852


In [10]:
df[df['Customer ID'].isna()].head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.00,NaN,United Kingdom
283,489463,71477,short,-240,2009-12-01 10:52:00,0.00,NaN,United Kingdom
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.00,NaN,United Kingdom
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.00,NaN,United Kingdom
577,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.55,NaN,United Kingdom
578,489525,85227,SET/6 3D KIT CARDS FOR KIDS,1,2009-12-01 11:49:00,0.85,NaN,United Kingdom
1055,489548,22271,FELTCRAFT DOLL ROSIE,1,2009-12-01 12:32:00,2.95,NaN,United Kingdom
1056,489548,22254,FELT TOADSTOOL LARGE,12,2009-12-01 12:32:00,1.25,NaN,United Kingdom
1057,489548,22273,FELTCRAFT DOLL MOLLY,3,2009-12-01 12:32:00,2.95,NaN,United Kingdom
1058,489548,22195,LARGE HEART MEASURING SPOONS,1,2009-12-01 12:32:00,1.65,NaN,United Kingdom


In [11]:
df[df["Quantity"] < 0].head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,"16,321.00",Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,"16,321.00",Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,"16,321.00",Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,"16,321.00",Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,"16,321.00",Australia
183,C489449,21871,SAVE THE PLANET MUG,-12,2009-12-01 10:33:00,1.25,"16,321.00",Australia
184,C489449,84946,ANTIQUE SILVER TEA GLASS ETCHED,-12,2009-12-01 10:33:00,1.25,"16,321.00",Australia
185,C489449,84970S,HANGING HEART ZINC T-LIGHT HOLDER,-24,2009-12-01 10:33:00,0.85,"16,321.00",Australia
186,C489449,22090,PAPER BUNTING RETRO SPOTS,-12,2009-12-01 10:33:00,2.95,"16,321.00",Australia
196,C489459,90200A,PURPLE SWEETHEART BRACELET,-3,2009-12-01 10:44:00,4.25,"17,592.00",United Kingdom


In [12]:
df["Invoice"] = df["Invoice"].astype("str")
df[df["Invoice"].str.match("^\\d{6}$") == False]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,"16,321.00",Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,"16,321.00",Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,"16,321.00",Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,"16,321.00",Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,"16,321.00",Australia
...,...,...,...,...,...,...,...,...
524695,C538123,22956,36 FOIL HEART CAKE CASES,-2,2010-12-09 15:41:00,2.10,"12,605.00",Germany
524696,C538124,M,Manual,-4,2010-12-09 15:43:00,0.50,"15,329.00",United Kingdom
524697,C538124,22699,ROSES REGENCY TEACUP AND SAUCER,-1,2010-12-09 15:43:00,2.95,"15,329.00",United Kingdom
524698,C538124,22423,REGENCY CAKESTAND 3 TIER,-1,2010-12-09 15:43:00,12.75,"15,329.00",United Kingdom


In [13]:
df["Invoice"].str.replace("[0-9]", "", regex=True).unique()

array(['', 'C', 'A'], dtype=object)

In [14]:
df[df["Invoice"].str.startswith("A")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,"-53,594.36",NaN,United Kingdom
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,"-44,031.79",NaN,United Kingdom
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,"-38,925.87",NaN,United Kingdom


In [15]:
df["StockCode"] = df["StockCode"].astype("str")
df[(df["StockCode"].str.match("^\\d{5}$") == False) & (df["StockCode"].str.match("^\\d{5}[a-zA-Z]+$") == False)]["StockCode"].unique()

array(['POST', 'D', 'DCGS0058', 'DCGS0068', 'DOT', 'M', 'DCGS0004',
       'DCGS0076', 'C2', 'BANK CHARGES', 'DCGS0003', 'TEST001',
       'gift_0001_80', 'DCGS0072', 'gift_0001_20', 'DCGS0044', 'TEST002',
       'gift_0001_10', 'gift_0001_50', 'DCGS0066N', 'gift_0001_30',
       'PADS', 'ADJUST', 'gift_0001_40', 'gift_0001_60', 'gift_0001_70',
       'gift_0001_90', 'DCGSSGIRL', 'DCGS0006', 'DCGS0016', 'DCGS0027',
       'DCGS0036', 'DCGS0039', 'DCGS0060', 'DCGS0056', 'DCGS0059', 'GIFT',
       'DCGSLBOY', 'm', 'DCGS0053', 'DCGS0062', 'DCGS0037', 'DCGSSBOY',
       'DCGSLGIRL', 'S', 'DCGS0069', 'DCGS0070', 'DCGS0075', 'B',
       'DCGS0041', 'ADJUST2', '47503J ', 'C3', 'SP1002', 'AMAZONFEE'],
      dtype=object)

In [16]:
df[df["StockCode"].str.contains("POST")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
89,489439,POST,POSTAGE,3,2009-12-01 09:28:00,18.00,"12,682.00",France
126,489444,POST,POSTAGE,1,2009-12-01 09:55:00,141.00,"12,636.00",USA
173,489447,POST,POSTAGE,1,2009-12-01 10:10:00,130.00,"12,362.00",Belgium
625,489526,POST,POSTAGE,6,2009-12-01 11:50:00,18.00,"12,533.00",Germany
927,C489538,POST,POSTAGE,-1,2009-12-01 12:18:00,9.58,"15,796.00",United Kingdom
...,...,...,...,...,...,...,...,...
522916,537967,POST,POSTAGE,1,2010-12-09 11:31:00,18.00,"12,494.00",France
522930,537995,POST,POSTAGE,2,2010-12-09 11:39:00,18.00,"12,712.00",Germany
522974,538003,POST,POSTAGE,8,2010-12-09 12:05:00,18.00,"12,429.00",Denmark
523064,538008,POST,POSTAGE,5,2010-12-09 12:17:00,18.00,"12,683.00",France


In [17]:
df[df["StockCode"].str.contains("D")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
197,C489459,90200D,PINK SWEETHEART BRACELET,-3,2009-12-01 10:44:00,4.25,"17,592.00",United Kingdom
204,C489459,90003D,CRYSTAL PAIR HEART HAIR SLIDES,-3,2009-12-01 10:44:00,3.75,"17,592.00",United Kingdom
208,C489459,90082D,DIAMANTE BOW BROOCH BLACK COLOUR,-2,2009-12-01 10:44:00,6.35,"17,592.00",United Kingdom
244,489462,90200D,PINK SWEETHEART BRACELET,3,2009-12-01 10:49:00,4.25,"17,592.00",United Kingdom
258,489462,90003D,CRYSTAL PAIR HEART HAIR SLIDES,3,2009-12-01 10:49:00,3.75,"17,592.00",United Kingdom
...,...,...,...,...,...,...,...,...
525147,538154,DOT,DOTCOM POSTAGE,1,2010-12-09 16:35:00,85.79,NaN,United Kingdom
525148,538155,84849D,HOT BATHS SOAP HOLDER,1,2010-12-09 16:52:00,1.69,"16,907.00",United Kingdom
525201,538156,17107D,"FLOWER FAIRY,5 SUMMER B'DRAW LINERS",10,2010-12-09 16:53:00,2.55,"15,555.00",United Kingdom
525325,538167,47591D,PINK FAIRY CAKE CHILDRENS APRON,2,2010-12-09 18:58:00,1.95,"14,713.00",United Kingdom


In [18]:
df[df["StockCode"].str.contains("DCGS0058")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
2377,489597,DCGS0058,MISO PRETTY GUM,1,2009-12-01 14:28:00,0.83,NaN,United Kingdom
8372,490074,DCGS0058,MISO PRETTY GUM,1,2009-12-03 14:39:00,0.83,NaN,United Kingdom
17264,490745,DCGS0058,MISO PRETTY GUM,1,2009-12-07 18:02:00,0.83,NaN,United Kingdom
30671,491969,DCGS0058,MISO PRETTY GUM,1,2009-12-14 17:57:00,0.83,NaN,United Kingdom
31652,491970,DCGS0058,MISO PRETTY GUM,1,2009-12-14 18:03:00,0.83,NaN,United Kingdom
32045,491971,DCGS0058,MISO PRETTY GUM,2,2009-12-14 18:37:00,0.83,NaN,United Kingdom
34668,492303,DCGS0058,MISO PRETTY GUM,1,2009-12-16 11:57:00,0.83,NaN,United Kingdom
37222,492425,DCGS0058,MISO PRETTY GUM,1,2009-12-16 17:58:00,0.83,NaN,United Kingdom
40878,492782,DCGS0058,MISO PRETTY GUM,1,2009-12-18 17:06:00,0.83,NaN,United Kingdom
41260,492783,DCGS0058,MISO PRETTY GUM,2,2009-12-18 17:15:00,0.83,NaN,United Kingdom


In [19]:
df[df["StockCode"].str.contains("DCGS0068")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
2378,489597,DCGS0068,DOGS NIGHT COLLAR,1,2009-12-01 14:28:00,8.65,NaN,United Kingdom
62734,494918,DCGS0068,DOGS NIGHT COLLAR,1,2010-01-19 17:49:00,8.47,NaN,United Kingdom


In [20]:
df[df["StockCode"].str.contains("DOT")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
2379,489597,DOT,DOTCOM POSTAGE,1,2009-12-01 14:28:00,647.19,NaN,United Kingdom
2539,489600,DOT,DOTCOM POSTAGE,1,2009-12-01 14:43:00,55.96,NaN,United Kingdom
2551,489601,DOT,DOTCOM POSTAGE,1,2009-12-01 14:44:00,68.39,NaN,United Kingdom
2571,489602,DOT,DOTCOM POSTAGE,1,2009-12-01 14:45:00,59.35,NaN,United Kingdom
2619,489603,DOT,DOTCOM POSTAGE,1,2009-12-01 14:46:00,42.39,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
524272,538071,DOT,DOTCOM POSTAGE,1,2010-12-09 14:09:00,885.94,NaN,United Kingdom
524887,538148,DOT,DOTCOM POSTAGE,1,2010-12-09 16:26:00,547.32,NaN,United Kingdom
525000,538149,DOT,DOTCOM POSTAGE,1,2010-12-09 16:27:00,620.68,NaN,United Kingdom
525126,538153,DOT,DOTCOM POSTAGE,1,2010-12-09 16:31:00,822.94,NaN,United Kingdom


In [21]:
df[df["StockCode"].str.contains("M")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1857,489594,79302M,"ART LIGHTS,FUNK MONKEY",1,2009-12-01 14:19:00,2.95,"15,005.00",United Kingdom
2697,489609,M,Manual,1,2009-12-01 14:50:00,4.00,NaN,United Kingdom
2845,489617,46000M,POLYESTER FILLER PAD 45x45cm,4,2009-12-01 14:59:00,1.55,"13,442.00",United Kingdom
3053,C489651,M,Manual,-1,2009-12-01 16:48:00,5.10,"17,804.00",United Kingdom
4366,489810,46000M,POLYESTER FILLER PAD 45x45cm,4,2009-12-02 12:47:00,1.55,"18,108.00",United Kingdom
...,...,...,...,...,...,...,...,...
521487,537823,16168M,FUNKY MONKEY GIFT BAG MEDIUM,1,2010-12-08 14:25:00,0.85,NaN,United Kingdom
522185,537850,16168M,FUNKY MONKEY GIFT BAG MEDIUM,24,2010-12-08 15:38:00,0.42,"16,722.00",United Kingdom
524696,C538124,M,Manual,-4,2010-12-09 15:43:00,0.50,"15,329.00",United Kingdom
524776,538147,M,Manual,1,2010-12-09 16:11:00,15.00,"13,090.00",United Kingdom


In [22]:
df[df["StockCode"].str.contains("DCGS0004")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
8371,490074,DCGS0004,HAYNES CAMPER SHOULDER BAG,1,2009-12-03 14:39:00,17.35,NaN,United Kingdom
98951,498699,DCGS0004,NaN,-2,2010-02-22 12:32:00,0.00,NaN,United Kingdom
164723,504935,DCGS0004,HAYNES CAMPER SHOULDER BAG,1,2010-04-19 11:19:00,16.98,NaN,United Kingdom
401083,527789,DCGS0004,HAYNES CAMPER SHOULDER BAG,1,2010-10-19 11:10:00,16.98,NaN,United Kingdom


In [23]:
df[df["StockCode"].str.contains("DCGS0076")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
8373,490074,DCGS0076,SUNJAR LED NIGHT NIGHT LIGHT,1,2009-12-03 14:39:00,16.48,NaN,United Kingdom
16426,490741,DCGS0076,SUNJAR LED NIGHT NIGHT LIGHT,1,2009-12-07 17:56:00,16.48,NaN,United Kingdom
17265,490745,DCGS0076,SUNJAR LED NIGHT NIGHT LIGHT,1,2009-12-07 18:02:00,16.48,NaN,United Kingdom
20324,491043,DCGS0076,SUNJAR LED NIGHT NIGHT LIGHT,1,2009-12-09 09:51:00,16.48,NaN,United Kingdom
30673,491969,DCGS0076,SUNJAR LED NIGHT NIGHT LIGHT,1,2009-12-14 17:57:00,16.48,NaN,United Kingdom
69630,495574,DCGS0076,SUNJAR LED NIGHT NIGHT LIGHT,1,2010-01-25 17:36:00,16.13,NaN,United Kingdom
87575,497590,DCGS0076,SUNJAR LED NIGHT NIGHT LIGHT,1,2010-02-10 17:59:00,16.13,NaN,United Kingdom
118446,500695,DCGS0076,SUNJAR LED NIGHT NIGHT LIGHT,1,2010-03-09 12:48:00,16.13,NaN,United Kingdom
169935,505510,DCGS0076,SUNJAR LED NIGHT NIGHT LIGHT,2,2010-04-22 14:33:00,16.13,NaN,United Kingdom
212437,509972,DCGS0076,SUNJAR LED NIGHT NIGHT LIGHT,1,2010-05-26 13:34:00,16.13,NaN,United Kingdom


In [24]:
df[df["StockCode"].str.contains("C2")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
9292,490127,C2,CARRIAGE,1,2009-12-03 18:13:00,50.00,"14,156.00",EIRE
14481,490541,C2,CARRIAGE,1,2009-12-07 09:25:00,50.00,NaN,EIRE
14502,490542,C2,CARRIAGE,1,2009-12-07 09:42:00,50.00,"14,911.00",EIRE
19541,490998,C2,CARRIAGE,1,2009-12-08 17:24:00,50.00,"16,253.00",United Kingdom
22803,491160,C2,CARRIAGE,1,2009-12-10 10:29:00,50.00,"14,911.00",EIRE
...,...,...,...,...,...,...,...,...
504361,536540,C2,CARRIAGE,1,2010-12-01 14:05:00,50.00,"14,911.00",EIRE
515057,537368,C2,CARRIAGE,1,2010-12-06 12:40:00,50.00,"14,911.00",EIRE
515390,537378,C2,CARRIAGE,1,2010-12-06 13:06:00,50.00,"14,911.00",EIRE
522913,537963,C2,CARRIAGE,1,2010-12-09 11:30:00,50.00,"13,369.00",United Kingdom


In [25]:
df[df["StockCode"].str.contains("BANK CHARGES")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
18410,C490943,BANK CHARGES,Bank Charges,-1,2009-12-08 14:08:00,15.00,"16,703.00",United Kingdom
18466,490948,BANK CHARGES,Bank Charges,1,2009-12-08 14:29:00,15.00,"16,805.00",United Kingdom
33435,C492206,BANK CHARGES,Bank Charges,-1,2009-12-15 16:32:00,848.43,NaN,United Kingdom
55948,C494438,BANK CHARGES,Bank Charges,-1,2010-01-14 12:15:00,767.99,NaN,United Kingdom
94431,498269,BANK CHARGES,Bank Charges,1,2010-02-17 15:03:00,15.00,"16,928.00",United Kingdom
...,...,...,...,...,...,...,...,...
450634,C532203,BANK CHARGES,Bank Charges,-1,2010-11-11 12:31:00,15.00,NaN,United Kingdom
455268,532637,BANK CHARGES,Bank Charges,1,2010-11-12 16:18:00,15.00,"13,174.00",United Kingdom
493162,535474,BANK CHARGES,Bank Charges,1,2010-11-26 13:56:00,15.00,"16,928.00",United Kingdom
507344,536779,BANK CHARGES,Bank Charges,1,2010-12-02 15:08:00,15.00,"15,823.00",United Kingdom


In [26]:
df[df["StockCode"].str.contains("DCGS0003")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
20249,491041,DCGS0003,BOXED GLASS ASHTRAY,1,2009-12-09 09:42:00,2.57,NaN,United Kingdom
232222,511858,DCGS0003,BOXED GLASS ASHTRAY,1,2010-06-11 10:05:00,2.51,NaN,United Kingdom
238355,512437,DCGS0003,BOXED GLASS ASHTRAY,1,2010-06-15 16:17:00,2.51,NaN,United Kingdom
244829,513099,DCGS0003,BOXED GLASS ASHTRAY,1,2010-06-21 15:13:00,2.51,NaN,United Kingdom
267418,515287,DCGS0003,BOXED GLASS ASHTRAY,1,2010-07-09 14:58:00,2.51,NaN,United Kingdom
421518,529727,DCGS0003,BOXED GLASS ASHTRAY,1,2010-10-29 14:12:00,2.51,NaN,United Kingdom
442200,531489,DCGS0003,BOXED GLASS ASHTRAY,1,2010-11-08 15:02:00,2.51,NaN,United Kingdom
466307,533348,DCGS0003,BOXED GLASS ASHTRAY,1,2010-11-17 09:23:00,2.51,NaN,United Kingdom
478086,534289,DCGS0003,BOXED GLASS ASHTRAY,1,2010-11-22 11:25:00,2.51,NaN,United Kingdom


In [27]:
df[df["StockCode"].str.contains("TEST001")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
27994,491725,TEST001,This is a test product.,10,2009-12-14 08:34:00,4.50,"12,346.00",United Kingdom
28251,491742,TEST001,This is a test product.,5,2009-12-14 11:00:00,4.50,"12,346.00",United Kingdom
28254,491744,TEST001,This is a test product.,5,2009-12-14 11:02:00,4.50,"12,346.00",United Kingdom
39398,492718,TEST001,This is a test product.,5,2009-12-18 10:47:00,4.50,"12,346.00",United Kingdom
45228,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,"12,346.00",United Kingdom
45230,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,"12,346.00",United Kingdom
56117,494450,TEST001,This is a test product.,5,2010-01-14 13:50:00,4.50,"12,346.00",United Kingdom
66084,495295,TEST001,This is a test product.,5,2010-01-22 13:30:00,4.50,"12,346.00",United Kingdom
89084,497819,TEST001,This is a test product.,5,2010-02-12 14:58:00,0.00,"14,103.00",United Kingdom
89180,497843,TEST001,This is a test product.,5,2010-02-12 15:47:00,0.00,"14,827.00",United Kingdom


In [28]:
df[df["StockCode"].str.contains("gift_0001_80")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
30620,491968,gift_0001_80,NaN,2,2009-12-14 17:56:00,0.00,NaN,United Kingdom
31079,491969,gift_0001_80,Dotcomgiftshop Gift Voucher £80.00,1,2009-12-14 17:57:00,69.56,NaN,United Kingdom
56446,C494472,gift_0001_80,Dotcomgiftshop Gift Voucher £80.00,-1,2010-01-14 14:51:00,69.56,NaN,United Kingdom
298842,518488,gift_0001_80,NaN,10,2010-08-09 12:29:00,0.00,NaN,United Kingdom


In [29]:
df[df["StockCode"].str.contains("DCGS0072")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
30672,491969,DCGS0072,CAT CAMOUFLAGUE COLLAR,1,2009-12-14 17:57:00,5.17,NaN,United Kingdom
34669,492303,DCGS0072,CAT CAMOUFLAGUE COLLAR,2,2009-12-16 11:57:00,5.17,NaN,United Kingdom
41261,492783,DCGS0072,CAT CAMOUFLAGUE COLLAR,1,2009-12-18 17:15:00,5.17,NaN,United Kingdom


In [30]:
df[df["StockCode"].str.contains("gift_0001_20")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
32048,491971,gift_0001_20,Dotcomgiftshop Gift Voucher £20.00,2,2009-12-14 18:37:00,17.39,NaN,United Kingdom
40905,492782,gift_0001_20,Dotcomgiftshop Gift Voucher £20.00,1,2009-12-18 17:06:00,17.39,NaN,United Kingdom
45213,493404,gift_0001_20,Dotcomgiftshop Gift Voucher £20.00,1,2009-12-23 15:30:00,17.39,NaN,United Kingdom
75097,496247,gift_0001_20,Dotcomgiftshop Gift Voucher £20.00,1,2010-01-29 16:11:00,17.02,NaN,United Kingdom
96283,498443,gift_0001_20,Dotcomgiftshop Gift Voucher £20.00,1,2010-02-19 09:49:00,17.02,NaN,United Kingdom
153282,503940,gift_0001_20,Dotcomgiftshop Gift Voucher £20.00,1,2010-04-08 14:47:00,17.02,NaN,United Kingdom
181337,506577,gift_0001_20,Dotcomgiftshop Gift Voucher £20.00,1,2010-04-30 13:45:00,17.02,NaN,United Kingdom
195555,508077,gift_0001_20,Dotcomgiftshop Gift Voucher £20.00,1,2010-05-12 16:29:00,17.02,NaN,United Kingdom
254427,513946,gift_0001_20,NaN,5,2010-06-29 13:35:00,0.00,NaN,United Kingdom
265191,515013,gift_0001_20,Dotcomgiftshop Gift Voucher £20.00,1,2010-07-07 16:29:00,17.02,NaN,United Kingdom


In [31]:
df[df["StockCode"].str.contains("DCGS0044")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
33819,492226,DCGS0044,HANDZ-OFF CAR FRESHENER,1,2009-12-15 18:33:00,2.57,NaN,United Kingdom


In [32]:
df[df["StockCode"].str.contains("TEST002")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
39411,492722,TEST002,This is a test product.,1,2009-12-18 10:55:00,1.00,"12,346.00",United Kingdom
44614,493294,TEST002,NaN,1,2009-12-22 15:15:00,0.00,NaN,United Kingdom


In [33]:
df[df["StockCode"].str.contains("gift_0001_10")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
40904,492782,gift_0001_10,Dotcomgiftshop Gift Voucher £10.00,1,2009-12-18 17:06:00,8.69,NaN,United Kingdom
125922,501441,gift_0001_10,Dotcomgiftshop Gift Voucher £10.00,1,2010-03-16 13:44:00,8.51,NaN,United Kingdom
136604,502486,gift_0001_10,Dotcomgiftshop Gift Voucher £10.00,1,2010-03-24 17:43:00,8.51,NaN,United Kingdom
218754,510637,gift_0001_10,Dotcomgiftshop Gift Voucher £10.00,1,2010-06-02 11:54:00,8.51,NaN,United Kingdom
298848,518496,gift_0001_10,NaN,10,2010-08-09 12:31:00,0.00,NaN,United Kingdom
352994,523696,gift_0001_10,Dotcomgiftshop Gift Voucher £10.00,1,2010-09-23 14:47:00,8.51,NaN,United Kingdom
442201,531489,gift_0001_10,Dotcomgiftshop Gift Voucher £10.00,1,2010-11-08 15:02:00,8.51,NaN,United Kingdom


In [34]:
df[df["StockCode"].str.contains("gift_0001_50")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
41263,492783,gift_0001_50,Dotcomgiftshop Gift Voucher £50.00,1,2009-12-18 17:15:00,43.48,NaN,United Kingdom
75136,496249,gift_0001_50,NaN,3,2010-01-29 16:17:00,0.00,NaN,United Kingdom
298841,518491,gift_0001_50,NaN,10,2010-08-09 12:29:00,0.00,NaN,United Kingdom
376731,525755,gift_0001_50,Dotcomgiftshop Gift Voucher £50.00,1,2010-10-07 10:23:00,42.55,NaN,United Kingdom


In [35]:
df[df["StockCode"].str.contains("DCGS0066N")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
44245,493265,DCGS0066N,NAVY CUDDLES DOG HOODIE,1,2009-12-22 15:05:00,8.65,NaN,United Kingdom
44335,493268,DCGS0066N,NaN,1,2009-12-22 15:07:00,0.00,NaN,United Kingdom
44377,493269,DCGS0066N,NAVY CUDDLES DOG HOODIE,1,2009-12-22 15:07:00,8.65,NaN,United Kingdom
44609,493289,DCGS0066N,NaN,1,2009-12-22 15:13:00,0.00,NaN,United Kingdom


In [36]:
df[df["StockCode"].str.contains("gift_0001_30")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
45212,493404,gift_0001_30,Dotcomgiftshop Gift Voucher £30.00,1,2009-12-23 15:30:00,26.08,NaN,United Kingdom
54564,494380,gift_0001_30,NaN,5,2010-01-13 17:39:00,0.00,NaN,United Kingdom
54636,494381,gift_0001_30,Dotcomgiftshop Gift Voucher £30.00,1,2010-01-13 17:44:00,25.53,NaN,United Kingdom
66801,495343,gift_0001_30,Dotcomgiftshop Gift Voucher £30.00,1,2010-01-22 18:05:00,25.53,NaN,United Kingdom
142163,502872,gift_0001_30,Dotcomgiftshop Gift Voucher £30.00,1,2010-03-29 09:59:00,25.53,NaN,United Kingdom
164334,504933,gift_0001_30,Dotcomgiftshop Gift Voucher £30.00,1,2010-04-19 11:13:00,25.53,NaN,United Kingdom
174432,505888,gift_0001_30,Dotcomgiftshop Gift Voucher £30.00,1,2010-04-26 14:14:00,25.53,NaN,United Kingdom
174563,505935,gift_0001_30,NaN,3,2010-04-26 14:54:00,0.00,NaN,United Kingdom
181338,506577,gift_0001_30,Dotcomgiftshop Gift Voucher £30.00,1,2010-04-30 13:45:00,25.53,NaN,United Kingdom
186882,507177,gift_0001_30,Dotcomgiftshop Gift Voucher £30.00,1,2010-05-06 15:09:00,25.53,NaN,United Kingdom


In [37]:
df[df["StockCode"].str.contains("PADS")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
62299,494914,PADS,PADS TO MATCH ALL CUSHIONS,1,2010-01-19 17:04:00,0.00,"16,705.00",United Kingdom
74731,496222,PADS,PADS TO MATCH ALL CUSHIONS,1,2010-01-29 13:53:00,0.00,"13,583.00",United Kingdom
77702,496473,PADS,PADS TO MATCH ALL CUSHIONS,1,2010-02-01 15:38:00,0.00,"17,350.00",United Kingdom
79794,496643,PADS,PADS TO MATCH ALL CUSHIONS,1,2010-02-03 11:58:00,0.00,"13,408.00",United Kingdom
90798,497935,PADS,PADS TO MATCH ALL CUSHIONS,1,2010-02-15 10:47:00,0.00,"13,408.00",United Kingdom
97716,498562,PADS,PADS TO MATCH ALL CUSHIONS,1,2010-02-21 12:03:00,0.00,"15,182.00",United Kingdom
101718,499056,PADS,PADS TO MATCH ALL CUSHIONS,1,2010-02-24 13:46:00,0.00,"13,765.00",United Kingdom
104480,499399,PADS,PADS TO MATCH ALL CUSHIONS,1,2010-02-26 13:26:00,0.00,"14,459.00",United Kingdom
123947,501176,PADS,PADS TO MATCH ALL CUSHIONS,1,2010-03-15 11:00:00,0.00,"14,857.00",United Kingdom
156809,504332,PADS,PADS TO MATCH ALL CUSHIONS,1,2010-04-12 16:30:00,0.00,"12,671.00",Germany


In [38]:
df[df["StockCode"].str.contains("ADJUST")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
70975,495732,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:20:00,96.46,NaN,EIRE
70976,495733,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:21:00,68.34,"14,911.00",EIRE
70977,495735,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:22:00,201.56,"12,745.00",EIRE
70978,495734,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:22:00,205.82,"14,911.00",EIRE
70979,C495737,ADJUST,Adjustment by john on 26/01/2010 16,-1,2010-01-26 16:23:00,10.50,"16,154.00",United Kingdom
...,...,...,...,...,...,...,...,...
208670,C509474,ADJUST,Adjustment by Peter on 24/05/2010 1,-1,2010-05-24 11:13:00,72.45,"16,422.00",United Kingdom
208671,C509475,ADJUST,Adjustment by Peter on 24/05/2010 1,-1,2010-05-24 11:13:00,358.47,"17,364.00",United Kingdom
249672,513564,ADJUST2,Adjustment by Peter on Jun 25 2010,1,2010-06-25 14:15:00,300.13,"17,050.00",United Kingdom
249673,513566,ADJUST2,Adjustment by Peter on Jun 25 2010,1,2010-06-25 14:15:00,358.47,"17,364.00",United Kingdom


In [39]:
df[df["StockCode"].str.contains("gift_0001_40")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
75137,496248,gift_0001_40,NaN,3,2010-01-29 16:17:00,0.00,NaN,United Kingdom
298847,518492,gift_0001_40,NaN,10,2010-08-09 12:30:00,0.00,NaN,United Kingdom
299652,518579,gift_0001_40,Dotcomgiftshop Gift Voucher £40.00,1,2010-08-10 12:08:00,31.35,NaN,United Kingdom
407443,528382,gift_0001_40,Dotcomgiftshop Gift Voucher £40.00,1,2010-10-21 18:01:00,34.04,NaN,United Kingdom


In [40]:
df[df["StockCode"].str.contains("gift_0001_60")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
75138,496250,gift_0001_60,NaN,3,2010-01-29 16:17:00,0.00,NaN,United Kingdom
298843,518490,gift_0001_60,NaN,10,2010-08-09 12:29:00,0.00,NaN,United Kingdom


In [41]:
df[df["StockCode"].str.contains("gift_0001_60")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
75138,496250,gift_0001_60,NaN,3,2010-01-29 16:17:00,0.00,NaN,United Kingdom
298843,518490,gift_0001_60,NaN,10,2010-08-09 12:29:00,0.00,NaN,United Kingdom


In [42]:
df[df["StockCode"].str.contains("gift_0001_70")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
75139,496251,gift_0001_70,NaN,2,2010-01-29 16:17:00,0.00,NaN,United Kingdom
235291,512168,gift_0001_70,Dotcomgiftshop Gift Voucher £70.00,1,2010-06-14 11:24:00,59.57,NaN,United Kingdom
298840,518489,gift_0001_70,NaN,10,2010-08-09 12:29:00,0.00,NaN,United Kingdom


In [43]:
df[df["StockCode"].str.contains("gift_0001_90")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
75141,496252,gift_0001_90,NaN,2,2010-01-29 16:18:00,0.00,NaN,United Kingdom
298839,518487,gift_0001_90,NaN,10,2010-08-09 12:28:00,0.00,NaN,United Kingdom


In [44]:
df[df["StockCode"].str.contains("DCGSSGIRL")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
80231,496722,DCGSSGIRL,NaN,-1,2010-02-03 14:04:00,0.00,NaN,United Kingdom
241372,512738,DCGSSGIRL,update,100,2010-06-17 14:11:00,0.00,NaN,United Kingdom
244831,513099,DCGSSGIRL,GIRLS PARTY BAG,3,2010-06-21 15:13:00,3.36,NaN,United Kingdom
246184,513200,DCGSSGIRL,GIRLS PARTY BAG,6,2010-06-22 16:28:00,3.36,NaN,United Kingdom
249938,513574,DCGSSGIRL,GIRLS PARTY BAG,1,2010-06-25 15:13:00,3.36,NaN,United Kingdom
251930,513655,DCGSSGIRL,GIRLS PARTY BAG,5,2010-06-28 10:02:00,3.36,NaN,United Kingdom
253997,513907,DCGSSGIRL,GIRLS PARTY BAG,1,2010-06-29 12:08:00,3.36,NaN,United Kingdom
385192,526405,DCGSSGIRL,GIRLS PARTY BAG,6,2010-10-11 13:53:00,3.36,NaN,United Kingdom
425616,530060,DCGSSGIRL,GIRLS PARTY BAG,2,2010-11-01 11:35:00,3.36,NaN,United Kingdom
427107,530140,DCGSSGIRL,GIRLS PARTY BAG,12,2010-11-01 16:45:00,3.36,NaN,United Kingdom


In [45]:
df[df["StockCode"].str.contains("DCGS0006")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
80337,496740,DCGS0006,NaN,-1,2010-02-03 14:28:00,0.00,NaN,United Kingdom


In [46]:
df[df["StockCode"].str.contains("DCGS0016")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
80338,496741,DCGS0016,NaN,-1,2010-02-03 14:29:00,0.00,NaN,United Kingdom


In [47]:
df[df["StockCode"].str.contains("DCGS0027")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
80339,496742,DCGS0027,NaN,-1,2010-02-03 14:29:00,0.00,NaN,United Kingdom


In [48]:
df[df["StockCode"].str.contains("DCGS0036")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
80342,496744,DCGS0036,NaN,-1,2010-02-03 14:35:00,0.00,NaN,United Kingdom


In [49]:
df[df["StockCode"].str.contains("DCGS0039")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
80361,496746,DCGS0039,NaN,-1,2010-02-03 14:36:00,0.00,NaN,United Kingdom


In [50]:
df[df["StockCode"].str.contains("DCGS0060")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
80953,496814,DCGS0060,NaN,-3,2010-02-04 09:46:00,0.00,NaN,United Kingdom


In [51]:
df[df["StockCode"].str.contains("DCGS0056")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
81143,496855,DCGS0056,NaN,-4,2010-02-04 11:46:00,0.00,NaN,United Kingdom


In [52]:
df[df["StockCode"].str.contains("DCGS0059")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
82846,496988,DCGS0059,NaN,-8,2010-02-05 11:21:00,0.00,NaN,United Kingdom


In [53]:
df[df["StockCode"].str.contains("GIFT")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
83304,497107,GIFT,NaN,-9,2010-02-05 15:18:00,0.00,NaN,United Kingdom


In [54]:
df[df["StockCode"].str.contains("DCGSLBOY")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
89000,497811,DCGSLBOY,NaN,-16,2010-02-12 14:45:00,0.00,NaN,United Kingdom


In [55]:
df[df["StockCode"].str.contains("m")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
44522,493283,79302m,"ART LIGHTS,FUNK MONKEY",1,2009-12-22 15:12:00,6.04,NaN,United Kingdom
96608,498492,m,Manual,1,2010-02-19 10:56:00,2.55,NaN,United Kingdom
96609,498492,m,Manual,1,2010-02-19 10:56:00,3.40,NaN,United Kingdom
113547,500264,79302m,"ART LIGHTS,FUNK MONKEY",1,2010-03-05 14:35:00,5.91,NaN,United Kingdom
143205,502974,79302m,"ART LIGHTS,FUNK MONKEY",1,2010-03-29 14:53:00,5.91,NaN,United Kingdom
154606,504111,79302m,"ART LIGHTS,FUNK MONKEY",1,2010-04-09 14:59:00,5.91,NaN,United Kingdom
157226,504396,m,Manual,1,2010-04-13 11:45:00,4.00,NaN,United Kingdom
228780,511509,m,Manual,1,2010-06-08 15:26:00,2.55,NaN,United Kingdom


In [56]:
df[df["StockCode"].str.contains("DCGS0053")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
98950,498698,DCGS0053,NaN,-2,2010-02-22 12:29:00,0.00,NaN,United Kingdom


In [57]:
df[df["StockCode"].str.contains("DCGS0062")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
98952,498700,DCGS0062,NaN,-1,2010-02-22 12:34:00,0.00,NaN,United Kingdom
414064,528952,DCGS0062,ROAD-RAGE CAR FRESHENER,1,2010-10-25 16:57:00,2.51,NaN,United Kingdom


In [58]:
df[df["StockCode"].str.contains("DCGS0037")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
98953,498701,DCGS0037,NaN,-1,2010-02-22 12:35:00,0.00,NaN,United Kingdom
236545,512289,DCGS0037,KEY-RING CORKSCREW,1,2010-06-14 14:49:00,12.72,NaN,United Kingdom


In [59]:
df[df["StockCode"].str.contains("DCGSSBOY")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
101692,499040,DCGSSBOY,NaN,-90,2010-02-24 13:16:00,0.00,NaN,United Kingdom
241371,512737,DCGSSBOY,update,100,2010-06-17 14:10:00,0.00,NaN,United Kingdom
244830,513099,DCGSSBOY,BOYS PARTY BAG,5,2010-06-21 15:13:00,3.36,NaN,United Kingdom
246183,513200,DCGSSBOY,BOYS PARTY BAG,7,2010-06-22 16:28:00,3.36,NaN,United Kingdom
249937,513574,DCGSSBOY,BOYS PARTY BAG,3,2010-06-25 15:13:00,3.36,NaN,United Kingdom
251929,513655,DCGSSBOY,BOYS PARTY BAG,4,2010-06-28 10:02:00,3.36,NaN,United Kingdom
278355,516470,DCGSSBOY,BOYS PARTY BAG,1,2010-07-20 15:28:00,3.36,NaN,United Kingdom
385191,526405,DCGSSBOY,BOYS PARTY BAG,1,2010-10-11 13:53:00,3.36,NaN,United Kingdom
425615,530060,DCGSSBOY,BOYS PARTY BAG,1,2010-11-01 11:35:00,3.36,NaN,United Kingdom
427106,530140,DCGSSBOY,BOYS PARTY BAG,4,2010-11-01 16:45:00,3.36,NaN,United Kingdom


In [60]:
df[df["StockCode"].str.contains("DCGSLGIRL")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
101693,499042,DCGSLGIRL,NaN,-57,2010-02-24 13:31:00,0.00,NaN,United Kingdom


In [61]:
df[df["StockCode"].str.contains("S")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
46,489437,84970S,HANGING HEART ZINC T-LIGHT HOLDER,12,2009-12-01 09:08:00,0.85,"15,362.00",United Kingdom
89,489439,POST,POSTAGE,3,2009-12-01 09:28:00,18.00,"12,682.00",France
122,489443,82001S,VINYL RECORD FRAME SILVER,24,2009-12-01 09:50:00,3.39,"14,110.00",United Kingdom
126,489444,POST,POSTAGE,1,2009-12-01 09:55:00,141.00,"12,636.00",USA
173,489447,POST,POSTAGE,1,2009-12-01 10:10:00,130.00,"12,362.00",Belgium
...,...,...,...,...,...,...,...,...
524199,538071,84971S,SMALL HEART FLOWERS HOOK,3,2010-12-09 14:09:00,1.66,NaN,United Kingdom
524238,538071,85199S,SMALL HANGING IVORY/RED WOOD BIRD,2,2010-12-09 14:09:00,0.85,NaN,United Kingdom
524271,538071,DCGS0076,SUNJAR LED NIGHT NIGHT LIGHT,1,2010-12-09 14:09:00,16.13,NaN,United Kingdom
524491,538093,POST,POSTAGE,5,2010-12-09 14:49:00,18.00,"12,682.00",France


In [62]:
df[df["StockCode"].str.contains("DCGS0069")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
123592,501161,DCGS0069,OOH LA LA DOGS COLLAR,1,2010-03-15 09:28:00,16.13,NaN,United Kingdom
148841,503549,DCGS0069,OOH LA LA DOGS COLLAR,1,2010-04-01 15:04:00,16.13,NaN,United Kingdom
151515,503823,DCGS0069,OOH LA LA DOGS COLLAR,1,2010-04-07 14:59:00,16.13,NaN,United Kingdom
405236,528145,DCGS0069,OOH LA LA DOGS COLLAR,1,2010-10-20 15:56:00,16.13,NaN,United Kingdom


In [63]:
df[df["StockCode"].str.contains("DCGS0070")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
148842,503549,DCGS0070,CAMOUFLAGE DOG COLLAR,1,2010-04-01 15:04:00,12.72,NaN,United Kingdom


In [64]:
df[df["StockCode"].str.contains("DCGS0075")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
148843,503549,DCGS0075,CAMOUFLAGUE DOG LEAD,1,2010-04-01 15:04:00,12.72,NaN,United Kingdom


In [65]:
df[df["StockCode"].str.contains("B")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
23,489436,35004B,SET OF 3 BLACK FLYING DUCKS,12,2009-12-01 09:06:00,4.65,"13,078.00",United Kingdom
42,489437,84507B,STRIPES DESIGN MONKEY DOLL,6,2009-12-01 09:08:00,2.55,"15,362.00",United Kingdom
62,489438,84031B,CHARLIE LOLA BLUE HOT WATER BOTTLE,56,2009-12-01 09:24:00,3.00,"18,102.00",United Kingdom
64,489438,84032B,CHARLIE + LOLA RED HOT WATER BOTTLE,56,2009-12-01 09:24:00,1.90,"18,102.00",United Kingdom
66,489438,84519B,CARROT CHARLIE+LOLA COASTER SET,60,2009-12-01 09:24:00,2.40,"18,102.00",United Kingdom
...,...,...,...,...,...,...,...,...
525187,538156,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-09 16:53:00,3.25,"15,555.00",United Kingdom
525269,538163,15060B,FAIRY CAKE DESIGN UMBRELLA,1,2010-12-09 17:27:00,3.75,"17,841.00",United Kingdom
525282,C538164,35004B,SET OF 3 BLACK FLYING DUCKS,-1,2010-12-09 17:32:00,1.95,"14,031.00",United Kingdom
525389,538170,85232B,SET OF 3 BABUSHKA STACKING TINS,2,2010-12-09 19:32:00,4.95,"13,969.00",United Kingdom


In [66]:
df[df["StockCode"].str.contains("DCGS0041")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
222345,511021,DCGS0041,HAYNES MINI-COOPER PLAYING CARDS,1,2010-06-04 16:13:00,5.06,NaN,United Kingdom


In [67]:
df[df["StockCode"].str.contains("ADJUST2")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
249672,513564,ADJUST2,Adjustment by Peter on Jun 25 2010,1,2010-06-25 14:15:00,300.13,"17,050.00",United Kingdom
249673,513566,ADJUST2,Adjustment by Peter on Jun 25 2010,1,2010-06-25 14:15:00,358.47,"17,364.00",United Kingdom
249674,513565,ADJUST2,Adjustment by Peter on Jun 25 2010,1,2010-06-25 14:15:00,72.45,"16,422.00",United Kingdom


In [68]:
df[df["StockCode"].str.contains("47503J")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
2981,489642,47503J,SET/3 FLORAL GARDEN TOOLS IN BAG,1,2009-12-01 16:19:00,7.95,NaN,United Kingdom
9784,490149,47503J,SET/3 FLORAL GARDEN TOOLS IN BAG,1,2009-12-04 09:43:00,16.48,NaN,United Kingdom
15894,490711,47503J,SET/3 FLORAL GARDEN TOOLS IN BAG,1,2009-12-07 15:26:00,7.95,"17,841.00",United Kingdom
20061,491041,47503J,SET/3 FLORAL GARDEN TOOLS IN BAG,1,2009-12-09 09:42:00,16.48,NaN,United Kingdom
24867,C491446,47503J,SET/3 FLORAL GARDEN TOOLS IN BAG,-7,2009-12-11 10:47:00,7.95,"14,191.00",United Kingdom
...,...,...,...,...,...,...,...,...
356535,524004,47503J,SET/3 FLORAL GARDEN TOOLS IN BAG,1,2010-09-26 16:06:00,7.95,"15,532.00",United Kingdom
496010,535803,47503J,SET/3 FLORAL GARDEN TOOLS IN BAG,6,2010-11-28 14:57:00,7.95,"14,410.00",United Kingdom
496295,535811,47503J,SET/3 FLORAL GARDEN TOOLS IN BAG,1,2010-11-28 15:18:00,7.95,"15,532.00",United Kingdom
509364,536945,47503J,SET/3 FLORAL GARDEN TOOLS IN BAG,1,2010-12-03 12:24:00,7.95,"14,083.00",United Kingdom


In [69]:
df[df["StockCode"].str.contains("C3")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
302041,518785,C3,NaN,-4999,2010-08-11 16:55:00,0.00,NaN,United Kingdom


In [70]:
df[df["StockCode"].str.contains("SP1002")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
377066,525772,SP1002,KID'S CHALKBOARD/EASEL,1,2010-10-07 11:12:00,2.95,"12,748.00",United Kingdom
377387,525837,SP1002,KID'S CHALKBOARD/EASEL,4,2010-10-07 12:23:00,2.95,"17,841.00",United Kingdom
426694,530135,SP1002,NaN,-27,2010-11-01 15:33:00,0.00,NaN,United Kingdom


In [71]:
df[df["StockCode"].str.contains("AMAZONFEE")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
440688,C531400,AMAZONFEE,AMAZON FEE,-1,2010-11-08 10:08:00,"6,706.71",NaN,United Kingdom
440698,531411,AMAZONFEE,AMAZON FEE,1,2010-11-08 10:11:00,"6,706.71",NaN,United Kingdom
517452,C537600,AMAZONFEE,AMAZON FEE,-1,2010-12-07 12:41:00,1.00,NaN,United Kingdom
517953,C537630,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:04:00,"13,541.33",NaN,United Kingdom
517955,537632,AMAZONFEE,AMAZON FEE,1,2010-12-07 15:08:00,"13,541.33",NaN,United Kingdom
519170,C537644,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:34:00,"13,474.79",NaN,United Kingdom
519251,C537647,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:41:00,"5,519.25",NaN,United Kingdom
519294,C537651,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:49:00,"13,541.33",NaN,United Kingdom
519295,C537652,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:51:00,"6,706.71",NaN,United Kingdom


## Notes

### Stock Code

* StockCode is meant to follow the pattern [0-9] {5} but seems to have legit values for [0-9]{5}[a-zA-Z]+

* Also contains other values:

| Code | Description | Action |
|---------------------|------------------------------------------------------------------------|-------------------------|
| DCGS | Looks valid, some quantities are negative though and customer ID is null | Exclude from clustering |
| D | Looks valid, represents discount values | Exclude from clustering |
| DOT | Looks valid, represents postage charges | Exclude from clustering |
| M or m | Looks valid, represents manual transactions | Exclude from clustering |
| C2 | Carriage transaction - not sure what this means | Exclude from clustering |
| C3 | Not sure, only 1 transaction | Exclude |
| BANK CHARGES or B | Bank charges | Exclude from clustering |
| S | Samples sent to customer | Exclude from clustering |
| TESTXXX | Testing data, not valid | Exclude from clustering |
| gift__XXX | Purchases with gift cards, might be interesting for another analysis, but no customer data | Exclude |
| PADS | Looks like a legit stock code for padding | Include |
| SP1002 | Looks like a special request item, only 2 transactions, 3 look legit, 1 has 0 pricing | Exclude for now |
| AMAZONFEE | Looks like fees for Amazon shipping or something | Exclude for now |
| ADJUSTX | Looks like manual account adjustments by admins | Exclude for now |

## Data cleaning

In [72]:
cleaned_df = df.copy()

In [73]:
cleaned_df["Invoice"] = cleaned_df["Invoice"].astype("str")

mask = (
    cleaned_df["Invoice"].str.match("^\\d{6}$") == True
)

cleaned_df = cleaned_df[mask]
cleaned_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,"13,085.00",United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,"13,085.00",United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,"13,085.00",United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,"13,085.00",United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,"13,085.00",United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,"17,530.00",United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,"17,530.00",United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,"17,530.00",United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,"17,530.00",United Kingdom


In [74]:
cleaned_df["StockCode"] = cleaned_df["StockCode"].astype("str")

mask = (
    (cleaned_df["StockCode"].str.match("^\\d{5}$") == True) 
    | (cleaned_df["StockCode"].str.match("^\\d{5}[a-zA-Z]+$") == True)
    | (cleaned_df["StockCode"].str.match("^PADS$") == True)
)
cleaned_df = cleaned_df[mask]
cleaned_df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,"13,085.00",United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,"13,085.00",United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,"13,085.00",United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,"13,085.00",United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,"13,085.00",United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,"17,530.00",United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,"17,530.00",United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,"17,530.00",United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,"17,530.00",United Kingdom


In [83]:
cleaned_df = cleaned_df.copy()
cleaned_df.dropna(subset=["Customer ID"], inplace=True)

In [76]:
cleaned_df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,"406,337.00",406337,"406,337.00","406,337.00"
mean,13.62,2010-07-01 10:11:06.543288320,2.99,"15,373.63"
min,1.00,2009-12-01 07:45:00,0.00,"12,346.00"
25%,2.00,2010-03-26 14:01:00,1.25,"14,004.00"
50%,5.00,2010-07-09 15:48:00,1.95,"15,326.00"
75%,12.00,2010-10-14 17:09:00,3.75,"16,814.00"
max,"19,152.00",2010-12-09 20:01:00,295.00,"18,287.00"
std,97.00,NaN,4.29,"1,677.37"


In [77]:
len(cleaned_df[cleaned_df["Price"] == 0])

28

In [78]:
cleaned_df = cleaned_df[cleaned_df["Price"] > 0.0]

In [79]:
cleaned_df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,"406,309.00",406309,"406,309.00","406,309.00"
mean,13.62,2010-07-01 10:14:25.869572352,2.99,"15,373.72"
min,1.00,2009-12-01 07:45:00,0.00,"12,346.00"
25%,2.00,2010-03-26 14:01:00,1.25,"14,006.00"
50%,5.00,2010-07-09 15:48:00,1.95,"15,326.00"
75%,12.00,2010-10-14 17:09:00,3.75,"16,814.00"
max,"19,152.00",2010-12-09 20:01:00,295.00,"18,287.00"
std,97.00,NaN,4.29,"1,677.33"


In [80]:
cleaned_df["Price"].min()

np.float64(0.001)

In [81]:
len(cleaned_df)/len(df)

0.7732429238325965

Dropped about 23% of records during data cleaning.